In [33]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import requests
import json
import io
import os
from dotenv import load_dotenv
from datetime import datetime

### Rainforest Product API

In [3]:
load_dotenv()

api_key = os.getenv('RAINFOREST_API_KEY')

In [ ]:
# set up the request parameters



params = {
    'api_key': api_key,
    'amazon_domain': 'amazon.com',
    'asin': 'B06Y5ZG66K',
    'type': 'product'
}

# make the http GET request to Rainforest API
api_result = requests.get('https://api.rainforestapi.com/request', params)

# print the JSON response from Rainforest API
json_result = json.dumps(api_result.json())

### Rainforest Search API

In [34]:
# set up the request parameters

def get_search_results(keyword, key=os.getenv('RAINFOREST_API_KEY')):
    params = {
        'api_key': key,
        'type': 'search',
        'amazon_domain': 'amazon.com',
        'search_term': keyword
    }

    # make the http GET request to Rainforest API
    api_result = requests.get('https://api.rainforestapi.com/request', params)

    # print the JSON response from Rainforest API
    search = api_result.json()
    
    return (search, keyword)

In [35]:
# Store retrieved data as JSON file
def store_json_search_results(results, keyword, current_timestamp=datetime.now()):
    date_string = current_timestamp.strftime("%Y-%m-%d")

    with open(f'data/raw/search_data_{keyword}_{date_string}.json', 'w') as json_file:
        json.dump(results, json_file, indent=4)
    
    print(f'{json_file} has been stored.')

In [17]:
# Retrieve stored JSON file

with open('data/raw/search_data_protein_powder_8.7.json', 'r') as raw_data:
    json_data = json.load(raw_data)

In [18]:
# Iterate through search results and store the values we care about

search_values = [
    (r.get('asin', 0), 
     r.get('position', 0),
     r.get('title', 0),
     r.get('link', 0),
     r.get('recent_sales', 0),
     r.get('categories', 0),
     r.get('unit_price', 0),
     r.get('rating', 0),
     r.get('is_prime', 0),
     r.get('ratings_total', 0),
     r.get('sponsored', 0),
     r.get('price', {}).get('value',0),
     r.get('amazons_choice', 0),
     r.get('bestseller', 0),
     r.get('coupon', 0)
    ) for r in json_data['search_results']]

In [20]:
# Store values as DataFrame for cleaning

search_df = pd.DataFrame(search_values, columns = [
    'asin', 
    'position', 
    'title', 
    'link', 
    'recent_sales', 
    'categories', 
    'unit_price', 
    'rating', 
    'is_prime', 
    'ratings_total', 
    'sponsored', 
    'price',
    'amazons_choice', 
    'bestseller', 
    'coupon'])

In [21]:
search_df.head()

,asin,position,title,link,recent_sales,categories,unit_price,rating,is_prime,ratings_total,sponsored,price,amazons_choice,bestseller,coupon
0,B08F3CC64W,1,"365 by Whole Foods Market, Protein Whey Grassf...",https://www.amazon.com/sspa/click?ie=UTF8&spc=...,400+ bought in past month,"[{'name': 'All Departments', 'id': 'aps'}]",$1.76$1.76/Ounce,4.2,True,119,0,18.99,0,0,0
1,B07VPKSD15,2,"GHOST Whey Protein Powder, Chips Ahoy - 2LB Tu...",https://www.amazon.com/sspa/click?ie=UTF8&spc=...,10K+ bought in past month,"[{'name': 'All Departments', 'id': 'aps'}]",$1.28$1.28/Ounce,4.5,True,11167,True,44.99,0,0,0
2,B0128RSQLC,3,BulkSupplements.com Grass Fed Whey Protein Pow...,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,1K+ bought in past month,"[{'name': 'All Departments', 'id': 'aps'}]",$1.08$1.08/Ounce,4.3,True,435,True,37.96,0,0,"{'badge_text': 'Extra $3.00 off', 'text': 'whe..."
3,B0BPJYDYSR,4,"FlavCity Protein Powder Smoothie, Chocolate Pe...",https://www.amazon.com/sspa/click?ie=UTF8&spc=...,7K+ bought in past month,"[{'name': 'All Departments', 'id': 'aps'}]",$1.59$1.59/Ounce,4.2,True,3064,True,59.99,0,0,"{'badge_text': 'Save 5%', 'text': 'with coupon'}"
4,B002DYIZH6,5,Optimum Nutrition Gold Standard 100% Whey Prot...,https://www.amazon.com/Optimum-Nutrition-Delic...,40K+ bought in past month,"[{'name': 'All Departments', 'id': 'aps'}]",$1.10$1.10/Ounce,4.6,True,211071,0,35.28,0,{'link': 'https://www.amazon.com/gp/bestseller...,"{'badge_text': 'Save 15%', 'text': 'with coupon'}"


In [26]:
search_df['per_unit_price'] = search_df['unit_price'].apply(lambda x: x.split('$')[1])

In [31]:
search_df['per_unit_price_measure'] = search_df['unit_price'].apply(lambda x: x.split('$')[-1].split('/')[1])

In [32]:
search_df

,asin,position,title,link,recent_sales,categories,unit_price,rating,is_prime,ratings_total,sponsored,price,amazons_choice,bestseller,coupon,per_unit_price,per_unit_price_measure
0,B08F3CC64W,1,"365 by Whole Foods Market, Protein Whey Grassf...",https://www.amazon.com/sspa/click?ie=UTF8&spc=...,400+ bought in past month,"[{'name': 'All Departments', 'id': 'aps'}]",$1.76$1.76/Ounce,4.2,True,119,0,18.99,0,0,0,1.76,Ounce
1,B07VPKSD15,2,"GHOST Whey Protein Powder, Chips Ahoy - 2LB Tu...",https://www.amazon.com/sspa/click?ie=UTF8&spc=...,10K+ bought in past month,"[{'name': 'All Departments', 'id': 'aps'}]",$1.28$1.28/Ounce,4.5,True,11167,True,44.99,0,0,0,1.28,Ounce
2,B0128RSQLC,3,BulkSupplements.com Grass Fed Whey Protein Pow...,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,1K+ bought in past month,"[{'name': 'All Departments', 'id': 'aps'}]",$1.08$1.08/Ounce,4.3,True,435,True,37.96,0,0,"{'badge_text': 'Extra $3.00 off', 'text': 'whe...",1.08,Ounce
3,B0BPJYDYSR,4,"FlavCity Protein Powder Smoothie, Chocolate Pe...",https://www.amazon.com/sspa/click?ie=UTF8&spc=...,7K+ bought in past month,"[{'name': 'All Departments', 'id': 'aps'}]",$1.59$1.59/Ounce,4.2,True,3064,True,59.99,0,0,"{'badge_text': 'Save 5%', 'text': 'with coupon'}",1.59,Ounce
4,B002DYIZH6,5,Optimum Nutrition Gold Standard 100% Whey Prot...,https://www.amazon.com/Optimum-Nutrition-Delic...,40K+ bought in past month,"[{'name': 'All Departments', 'id': 'aps'}]",$1.10$1.10/Ounce,4.6,True,211071,0,35.28,0,{'link': 'https://www.amazon.com/gp/bestseller...,"{'badge_text': 'Save 15%', 'text': 'with coupon'}",1.10,Ounce
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,B09YFHGK3F,66,"Creatine Monohydrate Powder 1 kg (2.2 Lbs), 50...",https://www.amazon.com/sspa/click?ie=UTF8&spc=...,1K+ bought in past month,"[{'name': 'All Departments', 'id': 'aps'}]",$0.91$0.91/Ounce,4.6,True,2588,0,31.95,0,0,0,0.91,Ounce
66,B09D667JS7,67,Gorilla Mode Pre Workout - Massive Pumps · Las...,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,3K+ bought in past month,"[{'name': 'All Departments', 'id': 'aps'}]",$2.72$2.72/Ounce,4.2,True,14008,0,59.99,0,0,0,2.72,Ounce
67,B0BPK1J1ZS,68,"Power Crunch PRO Protein Wafer Bars, Variety P...",https://www.amazon.com/sspa/click?ie=UTF8&spc=...,200+ bought in past month,"[{'name': 'All Departments', 'id': 'aps'}]",$0.98$0.98/Ounce,4.3,True,139,0,31.33,0,0,0,0.98,Ounce
68,B0CZ4JYB9M,69,Creatine Monohydrate Gummies Complex 6200mg fo...,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,10K+ bought in past month,"[{'name': 'All Departments', 'id': 'aps'}]",$0.17$0.17/Count,4.5,True,453,0,19.97,0,0,"{'badge_text': 'Extra 25% off', 'text': 'when ...",0.17,Count
